In [10]:
from urllib.request import urlopen
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup as bs
import re
import ssl
import sqlite3

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [26]:
conn = sqlite3.connect('/mnt/d/Users/Taina/Documents/Python/Goodreads.sqlite')
cur = conn.cursor()

cur.executescript('''CREATE TABLE IF NOT EXISTS Links (
    link  TEXT UNIQUE,
    retrieved INTEGER
)''')

conn.commit()

In [1]:
def robotParser(initial_url):
    
    rp = RobotFileParser()
    rp.set_url(initial_url + '/robots.txt')
    rp.read()
    
    return rp
    
def getLinks(page_url, initial_url, rp):
    
    #isso foi só para o link inicial, quando eu não tinha links na minha base de dados
    #para não dar erro quando eu atualizo para 1 embaixo
    cur.execute('''INSERT OR IGNORE INTO Links (link, retrieved) VALUES (?, 0)''', (page_url,))
    conn.commit()
    
    try:
        html = urlopen(initial_url + page_url, context = ctx)
    except:
        #usei o -1 para identificar links quebrados
        cur.execute('UPDATE Links SET retrieved = -1 WHERE link = ?', (page_url,))
        conn.commit()
        return
        
    soup = bs(html)
      
    for link in soup.findAll('a'):
        #ignorar itens sem link ou cujo link não seja do site goodreads
        if 'href' not in link.attrs or not (link.attrs['href'].startswith('/') or link.attrs['href'].startswith('https://www.goodreads.com/')):
            continue
            
        #padronizar os links que sobraram pra ficar somente o caminho, sem o goodreads inicial    
        link = link.attrs['href'].replace('https://www.goodreads.com','')
        
        #garantir que seja permitindo pro robot fazer o scrapping
        if not rp.can_fetch("*", link):
            continue
        
        #add link na minha base de dados, se ele já estiver na base não será adicionado
        cur.execute('''INSERT OR IGNORE INTO Links (link, retrieved) VALUES (?, 0)''', (link,))
        conn.commit()
    
    #marcando o link inicial como recuperado
    cur.execute('UPDATE Links SET retrieved = 1 WHERE link = ?', (page_url,))
    conn.commit()

initial_url = 'https://www.goodreads.com'

rp = robotParser(initial_url)

url = '/choiceawards/best-books-2019'

for i in range(20):
    
    print('retrieving from:', url)
    print(20-i, 'to go!\n')
    
    getLinks(url, initial_url, rp)
   
    try:
        cur.execute('SELECT link FROM Links WHERE retrieved = 0 LIMIT 1')
        url = cur.fetchone()[0]
        
    except:
        #teoricamente seria quando não tem mais nenhum link com status retrieved = 0
        print('all links retrieved!')
        conn.close()
        break

NameError: name 'RobotFileParser' is not defined

In [31]:
#não esquecer de fechar a conexão
conn.close()